In [11]:
import pymysql
import requests
import pandas as pd
import xml.etree.ElementTree as ET

# 데이터베이스 연결 파라미터
host = "211.191.187.151"
database = "slms"
user = "inpro"
password = "inpro0123"

def connect_to_database():
    try:
        # 데이터베이스 연결
        connection = pymysql.connect(
            host=host,
            user=user,
            password=password,
            database=database,
            port=3306,
            connect_timeout=10
        )
        print("데이터베이스 연결 성공")
        cursor = connection.cursor()
        return connection, cursor
    except pymysql.MySQLError as e:
        print(f"데이터베이스 연결 오류: {e}")
        return None, None

# farm_name을 별도로 정의
farm_name = input("목장명을 입력하세요: ")

# 연결 및 커서 가져오기
connection, cursor = connect_to_database()

if connection and cursor:
    try:
        # TBL_FARM에서 FARM_NUM, FARMER, PHONE 번호 조회
        query_farm_info = "SELECT FARM_NUM, FARMER, PHONE FROM TBL_FARM WHERE FARM_NAME = %s"
        cursor.execute(query_farm_info, (farm_name,))
        farm_info = cursor.fetchone()

        if farm_info:
            farm_num, farmer_name, phone_number = farm_info
            print(f"농장 '{farm_name}'의 FARM_NUM: {farm_num}, FARMER: {farmer_name}, PHONE: {phone_number}")

            # API URL
            url = "https://api.mtrace.go.kr/rest/myFarmData/farmUniqNoCattleBrdIndvd"
            payload = {
                "userId": "3148209264",
                "apiKey": "lstnfkQnFwmlPgpicMqF",
                "farmUniqueNo": farm_num,
                "farmerNm": farmer_name,
                "farmerHtelNo": phone_number
            }
            headers = {"Content-Type": "application/json"}

            try:
                # API 호출
                response = requests.post(url, json=payload, headers=headers)

                # 응답 확인
                if response.status_code == 200:
                    print("API 응답 성공!")
                    print("응답 내용:", response.text)

                    # XML 데이터 파싱
                    root = ET.fromstring(response.content)
                    animal_numbers = [
                        row.find('animalNo').text[3:]
                        for row in root.findall(".//row")
                        if row.find('animalNo') is not None
                    ]

                    if animal_numbers:
                        df_animal_numbers = pd.DataFrame(animal_numbers, columns=['animalNo'])
                        print("animalNo 데이터:")
                        print(df_animal_numbers)

                        # 텍스트 파일로 저장
                        output_file = "animal_numbers.txt"
                        with open(output_file, "w") as file:
                            for number in animal_numbers:
                                file.write(number + "\n")
                        print(f"animalNo 데이터가 '{output_file}' 파일로 저장되었습니다.")
                    else:
                        print("animalNo 데이터가 없습니다.")
                else:
                    print(f"API 요청 실패: {response.status_code}")
                    print("응답 내용:", response.text)

            except requests.exceptions.RequestException as e:
                print(f"API 요청 중 오류 발생: {e}")
        else:
            print(f"농장 '{farm_name}'에 대한 정보를 TBL_FARM에서 찾을 수 없습니다.")
    finally:
        # 연결 해제
        if connection and cursor:
            cursor.close()
            connection.close()
            print("데이터베이스 연결이 해제되었습니다.")
else:
    print("데이터베이스 연결에 실패했습니다.")


목장명을 입력하세요:  건영(신림)


데이터베이스 연결 성공
농장 '건영(신림)'의 FARM_NUM: 203893, FARMER: 이규만, PHONE: 010-8878-8905
API 응답 성공!
응답 내용: <?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>INFO_0000</resultCode><resultMsg>정상적으로 처리를 마쳤습니다.</resultMsg></header><data><row><farmUniqueNo>203893</farmUniqueNo><animalNo>410002144482431</animalNo></row><row><farmUniqueNo>203893</farmUniqueNo><animalNo>410002141725961</animalNo></row><row><farmUniqueNo>203893</farmUniqueNo><animalNo>410002166965404</animalNo></row><row><farmUniqueNo>203893</farmUniqueNo><animalNo>410002169438218</animalNo></row><row><farmUniqueNo>203893</farmUniqueNo><animalNo>410002176752050</animalNo></row><row><farmUniqueNo>203893</farmUniqueNo><animalNo>410002169433343</animalNo></row><row><farmUniqueNo>203893</farmUniqueNo><animalNo>410002168863848</animalNo></row><row><farmUniqueNo>203893</farmUniqueNo><animalNo>410002167934071</animalNo></row><row><farmUniqueNo>203893</farmUniqueNo><animalNo>410002142249505</animalNo></row><row>

## 축산물품질평가원에서 생년월일을 파악하여 월령을 구하기 위해 건영(신림)_code_result_df 파일을 건영(신림).txt로 만들어 축산물품질평가원에 이력제 데이터를 검색해서 

In [21]:
import pandas as pd

# 엑셀 파일 경로 설정
file_path = "C:/Users/alexkim/OneDrive - cnu.ac.kr/연구용역/스마트팜다부처패키지(2세대)/my works/파이썬/장기의사결정모형/젖소도태모델/비용예측모델/이력번호 조회_1732012662973.xlsx"

# 엑셀 파일 불러오기
df = pd.read_excel(file_path)

# '이력번호' 열에서 공백 제거 후 12자리 텍스트로 변환
df['이력번호'] = df['이력번호'].astype(str).str.replace(' ', '').str.zfill(12)

# 수정된 데이터를 같은 파일에 덮어쓰기
df.to_excel(file_path, index=False)

print(f"공백을 제거하고 12자리로 변환한 이력번호 데이터를 '{file_path}'에 저장했습니다.")


공백을 제거하고 12자리로 변환한 이력번호 데이터를 'C:/Users/alexkim/OneDrive - cnu.ac.kr/연구용역/스마트팜다부처패키지(2세대)/my works/파이썬/장기의사결정모형/젖소도태모델/비용예측모델/이력번호 조회_1732012662973.xlsx'에 저장했습니다.
